1. Suppose a student has taken an education load of size Rs 0.8 million. The
interest rate is 12%. Write a python program that generates the schedule of
repayment of the loan in 5 years (or ‘n’ number of years). Assume the first
payment date is 01 January 2026. Also, show the breakup of each payment is
principal and interest.

Step 1: Importing Neccesey libraries

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# Function to generate loan repayment schedule
def generate_loan_schedule(principal, annual_rate, years, start_date_str):
    monthly_rate = annual_rate / 12 / 100  # Monthly interest rate
    n_months = years * 12  # Total number of payments
    
    # Calculate EMI using the annuity formula
    emi = principal * monthly_rate * (1 + monthly_rate)**n_months / ((1 + monthly_rate)**n_months - 1)
    
    # Setup DataFrame
    schedule = []
    balance = principal
    start_date = datetime.strptime(start_date_str, "%d-%m-%Y")
    
    for i in range(1, n_months + 1):
        interest = balance * monthly_rate
        principal_payment = emi - interest
        balance -= principal_payment
        payment_date = start_date + pd.DateOffset(months=i-1)
        
        schedule.append({
            'Payment No': i,
            'Payment Date': payment_date.strftime("%d-%b-%Y"),
            'EMI': round(emi, 2),
            'Principal Paid': round(principal_payment, 2),
            'Interest Paid': round(interest, 2),
            'Outstanding Balance': round(balance if balance > 0 else 0, 2)
        })
    
    return pd.DataFrame(schedule)

In [6]:
df_schedule = generate_loan_schedule(
    principal=800000, 
    annual_rate=12, 
    years=5, 
    start_date_str="01-01-2026"
)

In [7]:
df_schedule.head(12)

,Payment No,Payment Date,EMI,Principal Paid,Interest Paid,Outstanding Balance
0,1,01-Jan-2026,17795.56,9795.56,8000.00,790204.44
1,2,01-Feb-2026,17795.56,9893.51,7902.04,780310.93
2,3,01-Mar-2026,17795.56,9992.45,7803.11,770318.48
3,4,01-Apr-2026,17795.56,10092.37,7703.18,760226.11
4,5,01-May-2026,17795.56,10193.30,7602.26,750032.81
5,6,01-Jun-2026,17795.56,10295.23,7500.33,739737.58
6,7,01-Jul-2026,17795.56,10398.18,7397.38,729339.40
7,8,01-Aug-2026,17795.56,10502.16,7293.39,718837.23
8,9,01-Sep-2026,17795.56,10607.19,7188.37,708230.05
9,10,01-Oct-2026,17795.56,10713.26,7082.30,697516.79


In [5]:
df_schedule.tail(5)

,Payment No,Payment Date,EMI,Principal Paid,Interest Paid,Outstanding Balance
55,56,01-Aug-2030,17795.56,16931.86,863.70,69437.65
56,57,01-Sep-2030,17795.56,17101.18,694.38,52336.47
57,58,01-Oct-2030,17795.56,17272.19,523.36,35064.28
58,59,01-Nov-2030,17795.56,17444.92,350.64,17619.36
59,60,01-Dec-2030,17795.56,17619.36,176.19,0.00


2. Consider a 20-years 8% bond with the coupon paid semi-annually. What will be
the present value of the bond? Use the following spot rates tables to compute
the present value. Also, compute duration and convexity.


### **Given Data**
| Period | Spot Rate (Annual) |
|--------|--------------------|
| 6M     | 2.90%              |
| 1 Yr   | 4.40%              |
| 2 Yr   | 4.80%              |
| 3 Yr   | 5.00%              |
| 5 Yr   | 5.30%              |
| 10 Yr  | 5.40%              |
| 20 Yr  | 5.50%              |

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

def interpolate_spot_rates():
    # Given annual spot rates (convert to semi-annual decimal)
    years = np.array([0.5, 1, 2, 3, 5, 10, 20])
    spot_rates = np.array([2.9, 4.4, 4.8, 5.0, 5.3, 5.4, 5.5]) / 100 / 2

    # Interpolate to get spot rate for every 6 months till 20 years (i.e. 40 periods)
    target_periods = np.arange(1, 41)  # 1 to 40 half-year periods
    target_years = target_periods / 2

    interpolate_func = interp1d(years, spot_rates, kind='linear', fill_value="extrapolate")
    interpolated_rates = interpolate_func(target_years)

    return interpolated_rates

def bond_pv_duration_convexity(face_value=100, coupon_rate_annual=8, years=20):
    spot_rates = interpolate_spot_rates()
    periods = years * 2
    coupon = (coupon_rate_annual / 2) * face_value / 100
    cash_flows = np.array([coupon] * (periods - 1) + [coupon + face_value])
    
    # Discount factors
    discount_factors = [(1 + r) ** (i+1) for i, r in enumerate(spot_rates)]
    pv_cash_flows = cash_flows / discount_factors
    pv_total = np.sum(pv_cash_flows)

    # Duration calculation
    time_periods = np.arange(1, periods + 1)
    weights = pv_cash_flows / pv_total
    macaulay_duration = np.sum(time_periods * weights) / 2  # in years

    # Convexity
    convexity = np.sum(weights * (time_periods * (time_periods + 1))) / (4 * (1 + spot_rates[0])**2)

    return pv_total, macaulay_duration, convexity

In [9]:
# Run the function
pv, duration, convexity = bond_pv_duration_convexity()
print(f"Present Value of Bond: ₹{pv:.2f}")
print(f"Macaulay Duration: {duration:.2f} years")
print(f"Convexity: {convexity:.2f}")


Present Value of Bond: ₹131.07
Macaulay Duration: 11.43 years
Convexity: 179.98


### 3. Complete the following tasks under this assignment.
   
a. Download one-year INFY, and RIL stock prices from National Stock
Exchange

b. Compute the daily returns for both.

c. Fit the return series separately to Generalized Gaussian distribution if you
can or fit the normal distribution.

d. Fit the T-Copula with the return series. You may use the `copulae`
package for the same or your own.

e. Construct a portfolio with equal units from both.

f. Simulate 10,000 or more random returns of your portfolio using copula

g. And calculate the maximum loss you may have in a day that you can say
with 95% confidence

a. Download 1-year INFY & RIL stock prices from NSE


In [21]:
# pip install yfinance


In [22]:
import yfinance as yf

start = "2024-04-01"
end = "2025-03-31"

infy = yf.download("INFY.NS", start=start, end=end)
ril = yf.download("RELIANCE.NS", start=start, end=end)

df = pd.DataFrame({
    "INFY": infy["Adj Close"],
    "RIL": ril["Adj Close"]
}).dropna()


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['INFY.NS']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['RELIANCE.NS']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [23]:
df

,INFY,RIL
Date,,


b. Compute Daily Returns

In [14]:
df = pd.DataFrame({
    "INFY": infy["Close"],
    "RIL": ril["Close"]
}).dropna()

returns = df.pct_change().dropna()

ValueError: attempt to get argmax of an empty sequence

In [15]:
infy

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
